In [1]:
import os
import numpy as np
import ceruleanml.data as data
import pandas as pd
from icevision.parsers import COCOMaskParser
from icevision.data import SingleSplitSplitter

parser = COCOMaskParser(annotations_filepath="./instances_slicks_test_v2.json", img_dir="./tiled_images_test_v2")

train_records = parser.parse(data_splitter=SingleSplitSplitter())

  0%|          | 0/1505 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/1188 [00:00<?, ?it/s]

In [2]:
from ceruleanml.coco_stats import region_props_for_instance_type
coco_class_list = ["infra_slick", "natural_seep", "coincident_vessel", "recent_vessel", "old_vessel", "ambiguous"]

cat_tables = []
for cat in coco_class_list:
    cat_table = region_props_for_instance_type(train_records[0], cat)
    cat_tables.append(cat_table)
instance_stats_df = pd.concat(cat_tables)
    

In [3]:
instance_stats_df.drop(columns = ["bbox-0","bbox-1","bbox-2","bbox-3", "img_name"]).groupby("category").mean().round()

,area,bbox_area,major_axis_length,axis_minor_length_bbox
category,,,,
ambiguous,4355.0,31540.0,233.0,109.0
coincident_vessel,1710.0,28898.0,227.0,96.0
infra_slick,9810.0,24566.0,174.0,90.0
natural_seep,4350.0,67715.0,324.0,171.0
old_vessel,2954.0,28886.0,249.0,97.0
recent_vessel,1514.0,26904.0,260.0,95.0


In [4]:
instance_stats_df.drop(columns = ["bbox-0","bbox-1","bbox-2","bbox-3", "img_name"]).groupby("category").max().round()

,area,bbox_area,major_axis_length,axis_minor_length_bbox
category,,,,
ambiguous,17586,205156,675.0,431
coincident_vessel,19358,227328,778.0,448
infra_slick,258059,262144,722.0,512
natural_seep,49779,262144,1099.0,512
old_vessel,40404,242688,796.0,475
recent_vessel,3900,147968,679.0,289


### Testing the area calculation for regionprops (since it differs from previous stats calculations of average area)


The record below has an area of 17586

In [4]:
record

BaseRecord

common: 
	- Record ID: 0
	- Image size ImgSize(width=512, height=512)
	- Filepath: tiled_images/S1A_IW_GRDH_1SDV_20200729T034859_20200729T034924_033664_03E6D3_93EF_vv-image_local_tile_0.png
	- Img: None
detection: 
	- masks: [<icevision.core.mask.RLE object at 0x7f7cfb3f1970>]
	- mask_array: None
	- Is Crowds: [1]
	- Areas: [17586]
	- BBoxes: [<BBox (xmin:256.0, ymin:275.0, xmax:512.0, ymax:512.0)>]
	- Class Map: <ClassMap: {'background': 0, 'infra_slick': 1, 'natural_seep': 2, 'coincident_vessel': 3, 'recent_vessel': 4, 'old_vessel': 5, 'ambiguous': 6}>
	- Labels: [6]

we can confirm this by taking the sum of labeled pixels

In [3]:
import skimage.io as skio

arr = skio.imread("tiled_images/S1A_IW_GRDH_1SDV_20200729T034859_20200729T034924_033664_03E6D3_93EF_vv-image_local_tile_0.png")

record = train_records[0][0]

record.as_dict()['detection']['masks'][0]

def create_mask_skimage_format(r, i):
    m = r["detection"]["masks"][i]
    return np.squeeze(m.to_mask(r["common"]["height"], r["common"]["width"]).data)
                      
(create_mask_skimage_format(record.as_dict(), 0) > 0).sum()

17586

regionprops table calculates this accurately

In [53]:
from skimage import measure
properties = ["area", "bbox_area", "major_axis_length", "bbox"]
image = create_mask_skimage_format(record.as_dict(), 0)
measure.regionprops_table(np.squeeze(image), properties=properties)

{'area': array([17586]),
 'bbox_area': array([60672]),
 'major_axis_length': array([     302.04]),
 'bbox-0': array([275]),
 'bbox-1': array([256]),
 'bbox-2': array([512]),
 'bbox-3': array([512])}

and this area is found int he summary dataframe

In [55]:
17586 in instance_stats_df["area"].unique()

True

In [ ]:
stats_df